In [13]:
# imports
from irsx.xmlrunner import XMLRunner

# from irsx import irsx_retrieve
import json
# import gmaps # visualization for map only
# import geopy
# from geopy.geocoders import GoogleV3
import pandas as pd
import googlemaps 
import numpy as np
import urllib, json, requests
import os
from pathlib import Path
# pd.set_option('display.float_format', '{:.2f}'.format)

In [12]:
from irsx.settings import IRSX_SETTINGS_LOCATION, IRSX_CACHE_DIRECTORY 
# IRSX_SETTINGS_LOCATION
# IRSX_CACHE_DIRECTORY
IRSX_CACHE_DIRECTORY = 'C:\\Users\\jared\Documents\\NW\\1_courses\\6_498_Capstone\\0_final_project\\2_foundations\\data'
# IRSX_CACHE_DIRECTORY
IRSX_WORKING_DIRECTORY = 'C:\\Users\\jared\Documents\\NW\\1_courses\\6_498_Capstone\\0_final_project\\2_foundations\\data\\filings'
IRSX_WORKING_DIRECTORY

'C:\\Users\\jared\\Documents\\NW\\1_courses\\6_498_Capstone\\0_final_project\\2_foundations\\data\\filings'

In [14]:
datapath = os.getcwd()
print(datapath)

##### convert NaNs to 0 for all columns - just so we can have proper dtypes and not deal with floats for ID columns.
##### Issue: all the NaN rows get filled out as well
# def conv(val):
#     if val == np.nan:
#         return 0 # or whatever else you want to represent your NaN with
#     return val

# # convert NaNs to 0 for all columns - just so we can have proper dtypes and not deal with floats for ID columns
# index_data = pd.read_csv('data/index_2022.csv', converters = {k: conv for k in range(9)}).dropna()

index_data = pd.read_csv('data/index_2022.csv').dropna()
index_data = index_data.fillna(0)


# int_cols = ['RETURN_ID', 'EIN', 'DLN', 'OBJECT_ID']
# for col in int_cols:
#     try:
#         index_data[col] = index_data[col].astype(int)
#     except:
#         print(col)

index_data['DLN']

C:\Users\jared\Documents\NW\1_courses\6_498_Capstone\0_final_project\2_foundations


0         93493013001462
1         93493013007062
2         93493013004162
3         93493013016712
4         93493013009212
               ...      
628590    93492315006132
628591    93492315012182
628592    93492315012332
628593    93492315013282
628594    93492315013432
Name: DLN, Length: 625304, dtype: int64

In [47]:
# foundationNameList = ["BILL AND MELINDA GATES FOUNDATION", "William and Flora Hewlett Foundation",\
#                       "Ford Foundation","Open Society Foundation", "Children's Investment Fund Foundation",\
#                       "The Mccarthur Foundation", "Conrad Hilton Foundation", "Rockefeller Foundation",\
#                       "Gordon and Betty Moore Foundation", "Silicon Valley Community Foundation","The Andrew Mellon Foundation",\
#                       "The Packard Foundation", "The Coca Cola Foundation","The Howard G.Buffet Foundation",\
#                       "Citi Group Foundation","The Seattle Internatonal Foundation","The Atlantic Philanthropies", \
#                       "Oak Foundation","Marine County Foundation", "Gulf Coast Community Foundation",\
#                       "Community Foundation of Collier County","Conservation Land Trust","Tinker Foundation",\
#                       "The Overbrook Foundation","Christensen Fund","Amgen Foundation","The Annenberg Foundation",\
#                       "BP Foundation","Carnegie Corporation of New York","The F.B. Heron Foundation",\
#                       "Firelight Foundation","Foundation Center","GE Foundation","The Kresge Foundation",\
#                       "The Lemelson Foundation","Levi Strauss Foundation","Rockefeller Brothers Fund",\
#                       "Skoll Foundation","W.K. Kellogg Foundation"]




class returnCounty():
    
    def __init__(self, XwalkFileHandle, gmapsobj):

        XWalkDF = pd.read_csv(XwalkFileHandle, sep = ',')        
        XWalkDF['ZIP'] = XWalkDF['ZIP'].astype(str)
        XWalkDF['ZIP'] = XWalkDF['ZIP'].str.zfill(5)

        XWalkDF['STCOUNTYFP'] = XWalkDF['STCOUNTYFP'].astype(str)
        XWalkDF['STCOUNTYFP'] = XWalkDF['STCOUNTYFP'].str.zfill(5)        
        
        self.XWalkDF = XWalkDF
        
        ##dupZipList is used to flag those zips which don't uniquely match to a county
        self.dupZipList = list( XWalkDF[XWalkDF.duplicated( ['ZIP'], keep=False)]['ZIP'] )
        self.gmapsobj = gmapsobj
        
        ## XWalkCountyFips is for matching by county name & state
        XWalkCountyFips = self.XWalkDF[['COUNTYNAME', 'STATE', 'STCOUNTYFP']].groupby(['STATE', 'COUNTYNAME']).agg(['min','max'])
        XWalkCountyFips.columns = XWalkCountyFips.columns.droplevel()
        XWalkCountyFips = XWalkCountyFips.loc[(XWalkCountyFips['max'] == XWalkCountyFips['min'])].drop('min', axis = 1).rename(columns={'max': 'STCOUNTYFP'})
        self.XWalkCountyFips = XWalkCountyFips.reset_index()

    def returnFipsLatLong(self, lat, long):
        ## Unused
        with urllib.request.urlopen("https://geo.fcc.gov/api/census/area?lat=42.3295&lon=-71.0826&format=json") as url:
            data = json.loads(url.read().decode())
            print(data)

        print(json.dumps(data, indent=4, sort_keys=True))

        print(data['results'][0]['state_fips'])
        print(data['results'][0]['county_fips'])
        print(data['results'][0]['county_name'])
        
    def countyFromZipCross(self, addressDict):
        #Verify ZipCode Present 
        returnDict = {'county' : None, 'fips' : None}
        
        if addressDict.get('zipcode') is not None and addressDict.get('zipcode')[0:5] not in self.dupZipList:
            zipLoc = self.XWalkDF.loc[self.XWalkDF['ZIP'] == addressDict.get('zipcode')]
            if not zipLoc.empty: # if ziploc is not empty ie there is a match
                returnDict['county'] = zipLoc['COUNTYNAME'].values[0]
                returnDict['fips'] = zipLoc['STCOUNTYFP'].values[0]
            
        return returnDict
        
        
    
    def countyFromGoog(self, addressDict):
        returnDict = {'county' : None, 'fips' : None}
        ### address this construct from addressDict
        # if addressDict is full (none of the values are missing) then create current_address string for geocoding; otherwise return None
        if (not addressDict['city']) | (not addressDict['addLine']) | (not addressDict['state']) | (not addressDict['zipcode']):
            current_address = ''
        else:
            current_address = addressDict.get('addLine') + " " +  addressDict.get('city') + ", " + addressDict.get('state') + " " + addressDict.get('zipcode')[0:5]
            geocode_result = self.gmapsobj.geocode(current_address)

            if len(geocode_result) != 0:
                gm = geocode_result[0] # dictionary
                g1 = gm.get("address_components")

                g3 = gm.get("geometry").get('location')
                testLat = g3.get('lat')
                testLong = g3.get('lng')

                #self.returnFipsLatLong(testLat, testLong)
                
                for i in range(len(g1)):
                    if 'administrative_area_level_2' in g1[i].get('types'):
                        returnDict['county'] = g1[i].get('long_name')
                    elif 'administrative_area_level_1' in g1[i].get('types'):
                        state = g1[i].get('short_name')
                        
                totest = self.XWalkCountyFips.loc[(self.XWalkCountyFips.STATE == addressDict.get('state')) 
                                                  & (self.XWalkCountyFips.COUNTYNAME == returnDict['county'])]['STCOUNTYFP'].values
                if len(totest) == 1:
                    returnDict['fips'] = totest[0]
        
        return returnDict

    
    def countyFromReverseGeoCode(self, addressDict):
        ### TBD -- let's see how many fail before caring about this.
        ### Basic idea: look up lat/long from address and then put lat/long and get list of possible addresses
        ### go through possible addrsses returned and get county

        foundCounty = False
        ### TODO:  find better way to throttle this: threading or some other way
        #    50 requests per second, and 100,000 per day
        if(countOfAddresses <10):

            geocode_result = my_gmaps.geocode(current_address)
        
        
        ### The code below was used when getting lat / long

        gm = geocode_result[0] # dictionary
        g1 = gm.get("address_components")
        g2 = gm.get("formatted_address")
        g3 = gm.get("geometry").get('location')
        curr_lat = g3.get('lat')
        curr_long = g3.get('lng')
        curr_county = ''
        print(str(curr_lat) + " and " + str(curr_long))

        for i in range(len(g1)):
#                             # this doesn't work if the county does not have county in the name e.g. New Orleans Parish
#                             if "County" in g1[i].get('long_name'):
#                                 curr_county = g1[i].get('long_name')
#                                 print(curr_county)
#                                 foundCounty = True

            typeArr = g1[i].get('types')
            # this area level appears to be the correct one, but still missing some US addresses
            if 'administrative_area_level_2' in typeArr:
                curr_county = g1[i].get('long_name')
                print(curr_county)
                foundCounty = True

        # TODO: find better way to deal with this: if county not in the list, reverse geocode
        if not foundCounty:
                countOfUSAddresses += 1 # increase count of US addresses that are missing counties
                reverseGeocoding = my_gmaps.reverse_geocode((curr_lat, curr_long)) # return array of estimates
                rg_length = len(reverseGeocoding)
                print(rg_length)
                for i in range(rg_length):
                    print(str(i)*20)
                    print(reverseGeocoding[i])
                    print(str(i)*20)

        latsLongs.loc[len(latsLongs)] = {'lat': curr_lat, 'long': curr_long, 'county': curr_county, 'state': state,'grantedAmt': contributionAmount, 'hasFunding': 1}
        
    def returnCountyFips(self, AddressDict):
        ### Basic Logic of this file is that it it attempts to grab the county name from the CrossWalk File (from Zip)
        ### If that fails, try to get it from google
        
        testDict = self.countyFromZipCross(AddressDict)
        if testDict.get('fips') is None:            
            testDict = self.countyFromGoog(AddressDict)
        
        return testDict

def findValid(pathToFolder):
    valid, invalid = 0,0
    valid_foundation = []

    foundInfo = {}

    for foundationName in [x.upper() for x in foundationNameList]:

        billsNum = getFoundationOID(pathToFolder,  foundationName)
        if billsNum is None:
            invalid += 1
            print('Invalid: %s' % foundationName)
        else:
            valid += 1
            valid_foundation.append(foundationName)        
            print('Valid: %s' % foundationName)        
            schedule_list, forms = getForms(billsNum)
            foundInfo[foundationName] = forms
    return foundInfo



foundationNameList = ["BILL & MELINDA GATES FOUNDATION"]

def getFoundationOID(pathToFolder, foundationName):
    filePath = pathToFolder + 'index_2022.csv'
    data = pd.read_csv(filePath)
    
    data['OBJECT_ID'] = data['OBJECT_ID'].astype(str)
    
    foundationInfo = data.loc[data['TAXPAYER_NAME'] == foundationName]
    if foundationInfo.empty:
        return None
    return foundationInfo['OBJECT_ID'].get( foundationInfo['OBJECT_ID'].keys()[0])

def getForms(fileNumber):
    ### RETURN A LIST OF SCHEDULES and the forms
    xml_runner = XMLRunner()
    parsed_filing = xml_runner.run_filing(fileNumber) # full XML
#     parsed_filing = xml_runner.irsx_retrieve(2022)
    forms = parsed_filing.get_result()  # An array of parsed schedules
    schedule_list = parsed_filing.list_schedules() # an array of form names
    return (schedule_list, forms)



googlemaps_key = open("googlemaps_key.txt", "r")
gmap_key = googlemaps_key.read()
# print(googlemaps_key)
my_gmaps = googlemaps.Client(key=gmap_key)
datapath = os.getcwd()
# pathToFolder= Path(datapath).parent / 'data'
pathToFolder = 'data/'

# getForms(202213159339300321)
print(getFoundationOID(datapath + "/data/", "BILL & MELINDA GATES FOUNDATION")) #XML in 2022_TEOS_XML_11A

forms_returned = getForms(202213159339300321)
print("schedules: ", forms_returned[0])
# print("Forms retrieved: ", forms_returned[1])
print(forms_returned[1][1])
foundInfo = findValid('data/')          

XwalkFile = pathToFolder + 'ZIP-COUNTY-FIPS_2017-06.csv' # before, was set to open the file

rc = returnCounty(XwalkFile, my_gmaps)

rc

202213159339300321
schedules:  ['ReturnHeader990x', 'IRS990PF', 'IRS990ScheduleB']
{'schedule_name': 'IRS990PF', 'groups': {'PFCpGnsLssTxInvstIncm': [{'object_id': 202213159339300321, 'ein': '562618866', 'PrprtyDsc': 'RETIREMENT OF CHARITABLE USE FIXED ASSETS', 'HwAcqrdCd': 'P', 'AcqrdDt': '2020-01-01', 'SldDt': '2021-12-31', 'DprctnAmt': '93820', 'CstOrOthrBssAmt': '95938', 'GnOrLssAmt': '-2118', 'GnsMnsExcssOrLsssAmt': '-2118'}, {'object_id': 202213159339300321, 'ein': '562618866', 'PrprtyDsc': 'DISPOSITION OF PROGRAM RELATED INVESTMENTS', 'HwAcqrdCd': 'P', 'AcqrdDt': '2020-01-01', 'SldDt': '2021-12-31', 'GrssSlsPrcAmt': '559911248', 'CstOrOthrBssAmt': '75408916', 'GnOrLssAmt': '484502332', 'GnsMnsExcssOrLsssAmt': '484502332'}, {'object_id': 202213159339300321, 'ein': '562618866', 'PrprtyDsc': 'SALE OF CHARITABLE USE ASSETS', 'HwAcqrdCd': 'P', 'AcqrdDt': '2020-01-01', 'SldDt': '2021-12-31', 'GrssSlsPrcAmt': '52776', 'GnOrLssAmt': '52776', 'GnsMnsExcssOrLsssAmt': '52776'}, {'object_id

Valid: BILL & MELINDA GATES FOUNDATION


In [48]:
returnDF = pd.DataFrame(columns=['foundation', 'granteeName','county', 'state', 'zipCode', 'grantedAmt', 'addressLine1', 'fips'])

### This loops through each and assigns a county when possible
for i,k in enumerate([x for x in foundInfo if foundInfo[x] is not None]):
    print(k)
    print(i)
    for form in foundInfo[k]:
        if 'IRS990PF' == form['schedule_name']:
            grantees = form['groups'].get('PFGrntOrCntrbtnPdDrYr')
            print("form was found")
            for grantee in grantees:
                # Grantee can appear in a number of different places, we take the first non-empty biz name
                #if businessName is empty, use  possibleBusinessName
                granteeName = ''
                
                if not grantee.get('RcpntBsnssNm_BsnssNmLn1Txt'): #if businessName is empty, use  possibleBusinessName
                    granteeName = grantee.get('GrntOrCntrbtnPdDrYr_RcpntPrsnNm')
                else:
                    granteeName = grantee.get('RcpntBsnssNm_BsnssNmLn1Txt')

                # For contribution Amount, set to -1 if empty
                contributionAmount = -1 if grantee.get('GrntOrCntrbtnPdDrYr_Amt')  == '' else grantee.get('GrntOrCntrbtnPdDrYr_Amt')
                
                ## Dealing with Address
                currentAddress = {'addLine' : grantee.get('RcpntUSAddrss_AddrssLn1Txt')
                                  , 'city' : grantee.get('RcpntUSAddrss_CtyNm')
                                  , 'state': grantee.get('RcpntUSAddrss_SttAbbrvtnCd')
                                  , 'zipcode': None if grantee.get('RcpntUSAddrss_ZIPCd') is None else grantee.get('RcpntUSAddrss_ZIPCd')[0:5]}
                                
                if currentAddress['zipcode'] is not None or (currentAddress['city'] is not None and currentAddress['state'] is not None):
                    K = rc.returnCountyFips(currentAddress)
                    county = K['county']
                    fips = K['fips']
                else:
                    county,fips = None, None

                returnDF.loc[len(returnDF)] = {'foundation' : k, 'granteeName': granteeName,'county': county \
                                               ,'state': currentAddress['state'], 'zipCode': currentAddress['zipcode']
                                               , 'grantedAmt': contributionAmount
                                               , 'addressLine1' : currentAddress['addLine']
                                              , 'fips': fips} 

returnDF['grantedAmt'] = returnDF.grantedAmt.astype(float)
returnDF.to_csv('Grantee_data.csv')
returnDF

BILL & MELINDA GATES FOUNDATION
0
form was found


,foundation,granteeName,county,state,zipCode,grantedAmt,addressLine1,fips
0,BILL & MELINDA GATES FOUNDATION,100 BLACK MEN OF AMERICA INC,Fulton County,GA,30303,500000.0,141 AUBURN AVENUE,13121
1,BILL & MELINDA GATES FOUNDATION,19TH NEWS THE,Travis County,TX,78746,203846.0,3267 BEE CAVES ROAD SUITE 107-353,48453
2,BILL & MELINDA GATES FOUNDATION,2164 INC,New York County,NY,10002,81780.0,445 PARK AVENUE SUITE 16A,36061
3,BILL & MELINDA GATES FOUNDATION,74 MEDIA INC THE,New York County,NY,10038,349995.0,222 BROADWAY 19TH FLOOR,36061
4,BILL & MELINDA GATES FOUNDATION,A-ALPHA BIO INC,King County,WA,98195,243987.0,FLUKE HALL SUITE 304 4000 MASON,53033
...,...,...,...,...,...,...,...,...
4108,BILL & MELINDA GATES FOUNDATION,ZEARN,New York County,NY,10001,100000.0,261 WEST 35TH STREET 15TH FLOOR,36061
4109,BILL & MELINDA GATES FOUNDATION,ZENYSIS TECHNOLOGIES INC,San Francisco County,CA,94107,168144.0,2325 3RD STREET SUITE 319,06075
4110,BILL & MELINDA GATES FOUNDATION,ZHONGZHI SOCIAL DEVELOPMENT PROMOTION CENTER,None,None,None,149070.0,None,None
4111,BILL & MELINDA GATES FOUNDATION,ZOETIS BELGIUM SA,None,None,None,1353911.0,None,None
